In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
Npoints = 1000
Nclusters = 4
Npoints_per_cluster = Npoints//Nclusters
(C1_mean_x,C1_mean_y) = (5,5)
(C2_mean_x,C2_mean_y) = (-5,5)
(C3_mean_x,C3_mean_y) = (-5,-5)
(C4_mean_x,C4_mean_y) = (5,-5)
C1_max_radius = 3
C2_max_radius = 3
C3_max_radius = 3
C4_max_radius = 3
clusterMeans = [(C1_mean_x,C1_mean_y),(C2_mean_x,C2_mean_y),(C3_mean_x,C3_mean_y),(C4_mean_x,C4_mean_y)]
clusterMaxRadius = [C1_max_radius,C2_max_radius,C3_max_radius,C4_max_radius]

for clusterNo in range(Nclusters):
    cluster_rad = clusterMaxRadius[clusterNo]*np.random.rand(Npoints_per_cluster,1)
    cluster_theta = 2*np.pi*np.random.rand(Npoints_per_cluster,1)
    cluster_mean = clusterMeans[clusterNo]
    cluster_x = cluster_mean[0] + (cluster_rad*np.cos(C1_theta))
    cluster_y = cluster_mean[1] + (cluster_rad*np.sin(C1_theta))
    if clusterNo>0:
        X = np.r_[X,cluster_x]
        Y = np.r_[Y,cluster_y]
    else:
        X = cluster_x
        Y = cluster_y

plt.scatter(X,Y)
plt.grid()
plt.title('Initial clusters')
plt.show()



In [ ]:
def update_cluster_estimates(X,Y,Nclusters,current_cluster_means_estimate):    
    Npoints = len(X)    
    clusters_estimate = np.zeros((Npoints,2,Nclusters))
    current_cluster_assignments = [0]*Npoints
    nextEntryId = [0]*Nclusters

    for n in range(Npoints):
        current_point = (X[n],Y[n])
        assignToClusterNo = findBestCluster(current_point,current_cluster_means_estimate)        
        id = nextEntryId[assignToClusterNo]
        nextEntryId[assignToClusterNo] +=1    
        current_cluster_assignments[n] = assignToClusterNo
        clusters_estimate[id,:,assignToClusterNo] = np.array(current_point).reshape(1,2)
    
    return clusters_estimate,current_cluster_assignments


In [ ]:
def findBestCluster(p,clusterMeans):
    nClusters = len(clusterMeans)
    currentMinDist = 1e9
    currentBestCluster = -1
    for i in range(nClusters):        
        currentClusterMean = clusterMeans[i]
        distFromCluster_i = (p[0]-currentClusterMean[0])**2 + (p[1]-currentClusterMean[1])**2
        if distFromCluster_i<currentMinDist:
            currentMinDist = distFromCluster_i
            currentBestCluster = i
    
    return currentBestCluster                

In [ ]:
def update_cluster_means(cluster_estimates):
    Nclusters = cluster_estimates.shape[2]
    updated_cluster_means_list=[]
    X = cluster_estimates[:,0]
    Y = cluster_estimates[:,1]
    X_mean = X.mean(axis=0).reshape(Nclusters,1)
    Y_mean = Y.mean(axis=0).reshape(Nclusters,1)
    updated_cluster_means = np.c_[X_mean,Y_mean]
    for c in range(Nclusters):
        updated_cluster_means_list.append((X_mean[c,0],Y_mean[c,0]))
        
    return updated_cluster_means_list

In [ ]:
def getNchanges(last_cluster_assignment,current_cluster_assignment):
    Npoints = len(last_cluster_assignment)
    nChanges = 0
    for i in range(Npoints):
        if last_cluster_assignment[i]!=current_cluster_assignment[i]:
            nChanges+=1
    return nChanges

In [ ]:
def showClusters(clusters):
    Nclusters = clusters.shape[2]
    for clusterNo in range(Nclusters):        
        cluster_x = updated_clusters[:,0,clusterNo]
        cluster_y = updated_clusters[:,1,clusterNo]
        if clusterNo>0:
            X_estimate = np.r_[X_estimate,cluster_x]
            Y_estimate = np.r_[Y_estimate,cluster_y]
        else:
            X_estimate = cluster_x
            Y_estimate = cluster_y

    plt.scatter(X_estimate,Y_estimate)
    plt.grid()
    plt.title('Estimates Clusters')
    plt.show()


In [ ]:
Nchanges = 1e2
iterationNo = 1
previous_cluster_assinments = []
# previous_means = clusterMeans
previous_means = [(0,1),(0,-1),(-1,-1),(1,-1)]
print(previous_means)
for k in range(Nclusters):
    start = k*Npoints_per_cluster
    end = (k+1)*Npoints_per_cluster
    assignValue = k
    for n in range(start,end):
        previous_cluster_assinments.append(assignValue)

while Nchanges>5:    
    (updated_clusters,updated_cluster_assignments) = update_cluster_estimates(X,Y,4,previous_means)    
    Nchanges = getNchanges(previous_cluster_assinments,updated_cluster_assignments)    
    print(f"Iteration {iterationNo} : Nchanges: {Nchanges}")
    updated_means = update_cluster_means(updated_clusters)
    print(f"Updated means: {updated_means}")
    previous_cluster_assinments = updated_cluster_assignments
    previous_means = updated_means
    
    
showClusters(updated_clusters)